In [3]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch

In [4]:
learning_rate = 1e-4
epochs = 20
batch_size = 32

In [68]:
dataset = dset.ImageFolder(root="dataset/",
                            transform=transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),      
                                transforms.Normalize((0.5, 0.5, 0.5),  
                                                    (0.5, 0.5, 0.5)), 
                            ]))

# dataset 분할
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print(len(train_dataset), len(test_dataset))

1100 276


C:\ProgramData\Anaconda3\envs\my_python_env\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [72]:

train_loader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)
                                


In [88]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 32)
        #    Pool     -> (?, 112, 112, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 112, 112, 32)
        #    Conv      ->(?, 112, 112, 64)
        #    Pool      ->(?, 56, 56, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        
        self.fc = torch.nn.Linear(56 * 56 * 64, 2, bias=True)

        
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [89]:
model = CNN()
criterion = torch.nn.CrossEntropyLoss()  # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [90]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 34


In [92]:
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. 
        
        
        X = X
        Y = Y

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.321884245
[Epoch:    2] cost = 0.123328425
[Epoch:    3] cost = 0.0792034119
[Epoch:    4] cost = 0.0549661443
[Epoch:    5] cost = 0.0478539839
[Epoch:    6] cost = 0.0419281013
[Epoch:    7] cost = 0.0279162508
[Epoch:    8] cost = 0.0243593007
[Epoch:    9] cost = 0.0220353678
[Epoch:   10] cost = 0.016770402
[Epoch:   11] cost = 0.0147329755
[Epoch:   12] cost = 0.0124267805
[Epoch:   13] cost = 0.0110055339
[Epoch:   14] cost = 0.0114991507
[Epoch:   15] cost = 0.00803379901
[Epoch:   16] cost = 0.00607385626
[Epoch:   17] cost = 0.00504310289
[Epoch:   18] cost = 0.00437060604
[Epoch:   19] cost = 0.0052151368
[Epoch:   20] cost = 0.00471530529


In [122]:
test_X = []
test_Y = []

for i in range(len(test_dataset)):
    test_X.append(test_dataset[i][0])
    test_Y.append(test_dataset[i][1])

print(test_Y)

[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [112]:
print(len(test_Y))
y = torch.zeros((276,2))
y[range(len(test_Y)), test_Y] = 1


276


In [136]:
with torch.no_grad():
    accuracy_list = []
    for X, Y in test_loader:
        X_test = X
        Y_test = Y
        prediction = model(X_test)    
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        accuracy_list.append(accuracy.item())
    a = np.array(accuracy_list)
    print('Accuracy:', np.mean(a))

Accuracy: 0.9765625
